In [1]:
from dotenv import load_dotenv

In [124]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders.base import BaseLoader
from langchain.schema import Document
import json
import re
from langchain.text_splitter import TextSplitter
import numpy as np
from typing import List
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
import re
# from langchain.text_splitter import SemanticChunker
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pickle
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import get_query_constructor_prompt
from langchain import hub
import json
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain import hub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [47]:
openai_api_key = os.getenv("OPENAI_API_KEY")

# 1. 문서 로드(Load Documents)

In [26]:
class JSONLLoader(BaseLoader):
    def __init__(self, file_path: str):
        self.file_path = file_path

    def load(self):
        documents = []
        seq_num = 1
        
        with open(self.file_path, 'r', encoding='utf-8') as file:
            for line in file:
                data = json.loads(line)
                doc = Document(
                    page_content=data['content'],
                    metadata={
                        'docid': data['docid'],
                        'src': data.get('src', ''),  # 'src' 필드가 없을 경우 빈 문자열 사용
                        'source': self.file_path,
                        'seq_num': seq_num,
                    }
                )
                documents.append(doc)
                seq_num += 1
        
        return documents

file_path = "/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl"
loader = JSONLLoader(file_path)
documents = loader.load()

print(f"문서의 수: {len(documents)}")

문서의 수: 4272


# 2. 문서 분할(Split Documents)

In [27]:
# RecursiveCharacterTextSplitter 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# 문서 분할
split_documents = text_splitter.split_documents(documents)

print(f"문서의 수: {len(documents)}")
print(f"분할된 문서의 수: {len(split_documents)}")

문서의 수: 4272
분할된 문서의 수: 4280


In [60]:
# 문장 단위 TextSplitter
class SentenceSplitter(TextSplitter):
    def split_text(self, text: str) -> list[str]:
        # 문장 끝 패턴: .!?로 끝나고 공백이 따라오는 경우
        # 줄바꿈 문자도 문장의 끝으로 간주
        return [sentence.strip() for sentence in re.split(r'(?<=[.!?])\s+|\n', text) if sentence.strip()]

# SentenceSplitter 초기화
sentence_splitter = SentenceSplitter()

# 문서 분할
sentence_split_documents = sentence_splitter.split_documents(documents)

print(f"원본 문서의 수: {len(documents)}")
print(f"분할된 문서의 수: {len(split_documents)}")

원본 문서의 수: 4272
분할된 문서의 수: 27964


In [50]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [134]:
# SemanticChunker
semantic_text_splitter = SemanticChunker(
    OpenAIEmbeddings(), add_start_index=True)

# documents를 split
semantic_split_documents = semantic_text_splitter.split_documents(documents)

print(f"원본 문서의 수: {len(documents)}")
print(f"분할된 문서의 수: {len(semantic_split_documents)}")

원본 문서의 수: 4272
분할된 문서의 수: 8540


In [135]:
# semantic_split_documents 저장
with open('semantic_split_documents.pkl', 'wb') as f:
    pickle.dump(semantic_split_documents, f)

In [136]:
# semantic_split_documents 로드
with open('semantic_split_documents.pkl', 'rb') as f:
    loaded_split_documents = pickle.load(f)

print(f"로드된 분할 문서의 수: {len(loaded_split_documents)}")

로드된 분할 문서의 수: 8540


# 3. 임베딩(Embedding) 및 벡터저장소 생성(Create Vectorstore)

In [65]:
# semantic_split_documents로 벡터저장소 생성 
vectorstore = FAISS.from_documents(documents=semantic_split_documents, embedding=OpenAIEmbeddings())

#>> FAISS.from_documents() 
#>> semantic_split_documents의 내용을 OpenAI 임베딩 모델을 통해 고차원 벡터로 변환 
#>> FAISS 인덱스 생성 
#>> 위에서 생성된 문서의 벡터를 FAISS 인덱스에 추가 

# 4. Retriever 생성

In [128]:
# 유사도 검색 테스트 
query = "나무의 분류에 대해 조사해 보기 위한 방법은?"

In [103]:
# OpenAI LLM 초기화 
llm = OpenAI(temperature=0)

In [130]:
# Standalone Query Generator 프롬프트 템플릿
standalone_query_prompt = PromptTemplate(
    input_variables=["question"],
    template="""질문 query를 요약하려고 합니다. 핵심내용을 포함한 주제를 출력해주세요.
    아래는 예시입니다. 
    
    원래의 질문: "금성이 밝게 보이는 이유가 뭐야?"
    생성할 standalone_query: "금성 밝기 원인"

    원래의 질문: {question}
    독립적인 질문:
    """
)

# Standalone Query Generator 체인 생성
standalone_query_chain = LLMChain(llm=llm, prompt=standalone_query_prompt)

# Standalone Query 생성
standalone_query = standalone_query_chain.run(query)

print(f"Original Query: {query}")
print(f"Standalone Query: {standalone_query}")

Original Query: 나무의 분류에 대해 조사해 보기 위한 방법은?
Standalone Query: "나무 분류 조사 방법"


In [123]:
# 생성된 Standalone Query를 사용하여 검색 (K*3개 검색)
K = 3
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": K*3})
search_result = retriever.get_relevant_documents(standalone_query)

# 중복 docid 제거 및 상위 3개 선택
unique_docs = []
seen_docids = set()

for doc in search_result:
    docid = doc.metadata.get('docid')
    if docid not in seen_docids:
        unique_docs.append(doc)
        seen_docids.add(docid)
        if len(unique_docs) == K:
            break

# 결과 출력
for i, doc in enumerate(unique_docs, 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content[:100]}...")  # 처음 100자만 출력
    print(f"메타데이터: {doc.metadata}")
    print("---")


문서 1:
내용: 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 ...
메타데이터: {'docid': 'c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d', 'src': 'ko_ai2_arc__ARC_Challenge__test', 'source': '/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl', 'seq_num': 2199, 'start_index': 98}
---

문서 2:
내용: 이러한 데이터를 정리하는 가장 좋은 방법은 날짜와 식물의 높이, 그리고 토양 유형을 함께 기록하는 것입니다. 이렇게 하면 나중에 데이터를 분석하거나 비교할 때 편리하고 정확한 결과...
메타데이터: {'docid': 'a153c822-be9f-4346-8558-34365ed7b4f0', 'src': 'ko_ai2_arc__ARC_Challenge__test', 'source': '/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl', 'seq_num': 1414, 'start_index': 69}
---

문서 3:
내용: 단풍나무는 가을에 아름다운 빨간색으로 변하는 나무로 유명합니다. 학생들은 이 나무의 특성을 배우고, 그 중에서도 미터 자로 측정할 수 있는 특성에 대해 알아봅니다. 미터 자는 길이...
메타데이터: {'docid': 'b730a81a-3903-42ca-9633-88b0ebb9eb42', 'src': 'ko_ai2_arc__ARC_Challenge__train', 'source': '/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl', 'seq_num': 474, 'start_index': 50}
---


# 5. 제출용 output 생성

In [131]:
def answer_question(query):
    # Standalone Query 생성
    standalone_query = standalone_query_chain.run(query)

    # 검색 수행 (K*3개 검색)
    K = 3
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": K*3})
    search_result = retriever.get_relevant_documents(standalone_query)

    # 중복 docid 제거 및 상위 3개 선택
    unique_docs = []
    seen_docids = set()

    for doc in search_result:
        docid = doc.metadata.get('docid')
        if docid not in seen_docids:
            unique_docs.append(doc)
            seen_docids.add(docid)
            if len(unique_docs) == K:
                break

    # RAG 프롬프트 가져오기
    rag_prompt = hub.pull("rlm/rag-prompt")

    # 첫 번째 문서의 내용만 사용
    context = unique_docs[0].page_content if unique_docs else ""

    # LLM 체인 생성
    llm = OpenAI(temperature=0)
    rag_chain = LLMChain(llm=llm, prompt=rag_prompt)

    # 답변 생성 (첫 번째 문서만 참고)
    answer = rag_chain.run(context=context, question=query)
    
    standalone_query = standalone_query_chain.run(query).strip('"')  # 따옴표 제거

    # topk 및 references 정보 추출
    topk = [doc.metadata.get('docid') for doc in unique_docs]
    references = [
        {
            "score": doc.metadata.get('score', 0),
            "content": doc.page_content
        }
        for doc in unique_docs
    ]

    return {
        "standalone_query": standalone_query,
        "topk": topk,
        "answer": answer,
        "references": references
    }

In [132]:
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            output = {
                "eval_id": j["eval_id"],
                "standalone_query": response["standalone_query"],
                "topk": response["topk"],
                "answer": response["answer"],
                "references": response["references"]
            }
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [116]:
# # OpenAI LLM 초기화 
# llm = OpenAI(temperature=0)

In [133]:
# 평가 실행
eval_rag("/data/ephemeral/home/upstage-ai-final-ir2/HM/data/eval.jsonl", "sample_submission_6.csv")

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
Answer:  나무의 분류를 조사하기 위한 방법은 나무들이 같은 속에 속해 있는지 확인하고, 같은 속에 속한 나무들이 유사한 특징을 가지고 있는지 살펴보는 것입니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 수 있습니다.

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer:  전세계의 공공 교육 지출은 세계 GDP의 약 4%를 차지하고 있습니다. 이는 국가들이 교육에 많은 자원을 투자하고 있다는 것을 보여줍니다.

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Answer:  기억 상실증은 대뇌의 기능에 문제가 생겼을 때 발생할 수 있습니다. 이것이 발생하는 원인은 다양하지만 대개는 뇌의 손상이나 질병으로 인해 발생합니다.

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]
Answer:  학교 버스는 학교와 학생들 사이의 거리를 줄이고, 편리한 교통수단을 제공하여 학생들의 교육에 도움을 줍니다. 또한, 학교 버스는 교통체증을 줄이고 환경 보호에도 기여합니다. 따라서, 통학 버스는 교육과 환경 측면에서 매우 중요한 가치를 가지고 있습니다.

Test 4
Question: [{'role': 'user', 'content': 'Dmitri Ivanovsky가 누구야?'}]
Answer:  Dmitri Ivanovsky는 연구자이며, 그의 연구는 나이에 따른 인지 능력의 변화를 이해